In [42]:
from abc import ABC, abstractmethod
import datetime, random, json

#--------------------------Account--------------------------------------------

class Account(ABC):
  def __init__(self, name, account_no, balance = 0, pin = 1998):
    self.owner = name
    self.account_no = account_no
    self.__balance = balance
    self.transaction = []
    self.__pin = pin

  def _get_balance(self):
    return self.__balance

  def _update_balance(self, amount):
    self.__balance += amount

  def _check_pin(self, pin):
    return self.__pin == pin


  def deposit(self, amount, pin= None, system = False):
    if not system:
      if not self._check_pin(pin):
        print("Invalid pin. Deposit denied")
        return

    if amount > 0:
      self.__balance += amount
      self.transaction.append(f"Deposit: +{amount} ||\tAvailable Balance is {self.__balance}")
      print(f"Deposit {amount}. New balance is {self.__balance}")
    else:
      print("amount should be positive")


  @abstractmethod
  def withdraw(self, amount, pin):
    pass


  def print_statement(self, pin):
    if not self._check_pin(pin):
      print("Invalid pin. view statement denied")
      return

    print(f"-------\nTransaction history for {self.owner}-------")
    for t in self.transaction:
      print(t)
    print(f"Final balance is {self.__balance}")


  def get_pin(self):
    return self.__pin



#-------------------------------Savings Account--------------------------------

class SavingsAccount(Account):
  def __init__(self, name, account_no, balance=0, pin= 1998):
    super().__init__(name, account_no, balance, pin)
    self.daily_limit = 50000
    self.daily_withdraw = 0
    self.last_withdraw_date = None

  def withdraw(self, amount, pin):
    today = datetime.date.today()

    if self.last_withdraw_date != today:
      self.daily_withdraw = 0
      self.last_withdraw_date = today

    if not self._check_pin(pin):
      print("Invalid pin. withdraw denied")
      return

    if self.daily_withdraw + amount <= self.daily_limit:
      if self._get_balance() >= amount:
        self.daily_withdraw += amount
        self._update_balance(-amount)
        self.transaction.append(f"withdraw: -{amount} || Remaing Balance is {self._get_balance()}")
        print(f"withdraw amount {amount}. Balance {self._get_balance()}")
      else:
        print("insufficient balance")
    else:
      print("daily withdraw limit reached")

  def calculate_interest(self):
    return self._get_balance() * 0.05


#-----------------------------Current Account-------------------------------

class CurrentAccount(Account):
  def withdraw(self, amount,pin):
    if not self._check_pin(pin):
      print("Invalid pin. withdraw denied")
      return

    fee = 10
    total = fee + amount
    if self._get_balance() + 3000 >= total:
      self._update_balance(-total)
      self.transaction.append(f"withdraw: -{amount} ||, Fee: -{fee} || Remaing Balance is {self._get_balance()}")
      print(f"withdraw amount {amount} || Balance {self._get_balance()}")
    else:
      print("overdraft limit exceeded in current account")

  def calculate_interest(self):
    return self._get_balance() * 0.02

#-----------------------------Business Account----------------------------

class BusinessAccount(CurrentAccount):
  def withdraw(self, amount,pin):
    if not self._check_pin(pin):
      print("Invalid pin. withdraw denied")
      return

    fee = 50
    total = fee + amount
    if self._get_balance() + 10000 >= total:
      self._update_balance(-total)
      self.transaction.append(f"withdraw: -{amount} ||, Fee: -{fee} || Remaing Balance is {self._get_balance()}")
      print(f"withdraw amount {amount}. Balance {self._get_balance()}")
    else:
      print("overdraft limit exceeded in current account")

  def calculate_interest(self):
    return self._get_balance() * 0.03


#--------------------------------Bank---------------------------------------

class Bank:
  def __init__(self,name, data_file = "bank_data.json"):
    self.name = name
    self.accounts = {}
    self.last_account_no = 1111
    self.data_file = data_file
    self.load_data()

  def save_data(self):
    data = {
        "last_account_no": self.last_account_no,
        "accounts": {}
    }
    for acc_no, acc in self.accounts.items():
        data["accounts"][acc_no] = {
            "type": acc.__class__.__name__,
            "owner": acc.owner,
            "balance": acc._get_balance(),
            "transactions": acc.transaction,
            "pin": acc.get_pin()
        }

    with open(self.data_file, "w") as f:
        json.dump(data, f, indent=4)


  def load_data(self):
    try:
      with open(self.data_file, "r") as f:
          data = json.load(f)
          self.last_account_no = data["last_account_no"]
          for acc_no, acc_data in data["accounts"].items():
              pin = acc_data.get("pin", random.randint(1000, 9999))
              if acc_data["type"] == "SavingsAccount":
                  acc = SavingsAccount(acc_data["owner"], int(acc_no), acc_data["balance"], pin)
              elif acc_data["type"] == "CurrentAccount":
                  acc = CurrentAccount(acc_data["owner"], int(acc_no), acc_data["balance"], pin)
              elif acc_data["type"] == "BusinessAccount":
                  acc = BusinessAccount(acc_data["owner"], int(acc_no), acc_data["balance"], pin)
              acc.transaction = acc_data["transactions"]
              self.accounts[int(acc_no)] = acc
    except FileNotFoundError:
      pass

  def generate_account_no(self):
    self.last_account_no += 1
    return self.last_account_no

  def create_account(self, owner, account_type, balance=0, pin = None):
    account_no = self.generate_account_no()
    if pin is None or len(str(pin)) != 4:
            pin = random.randint(1000, 9999)

    if account_type == "Savings":
      acc = SavingsAccount(owner, account_no, balance, pin)
    elif account_type == "Current":
      acc = CurrentAccount(owner, account_no, balance, pin)
    elif account_type == "Business":
      acc = BusinessAccount(owner,account_no, balance, pin)
    else:
      print("Invalid account type")
      return None

    self.accounts[account_no] = acc
    self.save_data()
    print(f"------Account Created Successfully-----: \nOwner:{owner} ||, Account type:{account_type} ||, Account number:{account_no},\npin of this account is {pin}")
    return acc



  def get_account(self,account_no):
    return self.accounts.get(account_no, None)

  def transfer(self, from_account_no, to_account_no, amount, pin):
    from_acc = self.get_account(from_account_no)
    to_acc = self.get_account(to_account_no)

    if not from_acc or not to_acc:
        print("Invalid account number")
        return

    if not from_acc._check_pin(pin):
        print("Invalid PIN. Transfer denied.")
        return

    old_balance = from_acc._get_balance()
    from_acc.withdraw(amount, pin)

    if from_acc._get_balance() < old_balance:
        to_acc.deposit(amount, system=True)
        self.save_data()
        print(f"Transferred {amount} from {from_acc.owner} to {to_acc.owner}")


  def apply_interest(self):
    for acc in self.accounts.values():
      interest = acc.calculate_interest()
      acc.deposit(interest, system = True)
      self.save_data()
      print(f"Applied {interest:.2f} interest to {acc.owner}'s account")




In [43]:
def main():
    bank = Bank("MyBank")

    while True:
        print("\n-------- Welcome to MyBank -------")
        print("1. Create Account")
        print("2. Deposit Money")
        print("3. Withdraw Money")
        print("4. Transfer Money")
        print("5. Show Statement")
        print("6. Apply Interest")
        print("7. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Enter owner name: ")
            acc_type = input("Enter account type (Savings/Current/Business): ")
            balance = float(input("Enter initial deposit: "))
            pin = int(input("Enter 4-digit PIN: "))
            bank.create_account(name, acc_type, balance, pin)

        elif choice == "2":
            acc_no = int(input("Enter account number: "))
            pin = int(input("Enter PIN: "))
            amount = float(input("Enter deposit amount: "))
            acc = bank.get_account(acc_no)
            if acc:
                acc.deposit(amount, pin)
                bank.save_data()
            else:
                print("Account not found!")

        elif choice == "3":
            acc_no = int(input("Enter account number: "))
            pin = int(input("Enter PIN: "))
            amount = float(input("Enter withdrawal amount: "))
            acc = bank.get_account(acc_no)
            if acc:
                acc.withdraw(amount, pin)
                bank.save_data()
            else:
                print("Account not found!")

        elif choice == "4":
            from_acc = int(input("Enter sender account number: "))
            to_acc = int(input("Enter receiver account number: "))
            pin = int(input("Enter sender's PIN: "))
            amount = float(input("Enter transfer amount: "))
            bank.transfer(from_acc, to_acc, amount, pin)

        elif choice == "5":
            acc_no = int(input("Enter account number: "))
            pin = int(input("Enter PIN: "))
            acc = bank.get_account(acc_no)
            if acc:
                acc.print_statement(pin)
            else:
                print("Account not found!")

        elif choice == "6":
            bank.apply_interest()

        elif choice == "7":
            print("Thank you for banking with us!")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


-------- Welcome to MyBank -------
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Transfer Money
5. Show Statement
6. Apply Interest
7. Exit
Enter your choice: 7
Thank you for banking with us!
